<a href="https://colab.research.google.com/github/SalimaMamma/HPC/blob/main/HPC_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

--2022-04-30 11:46:08--  https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Resolving developer.nvidia.com (developer.nvidia.com)... 152.195.19.142
Connecting to developer.nvidia.com (developer.nvidia.com)|152.195.19.142|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 [following]
--2022-04-30 11:46:09--  https://developer.nvidia.com/compute/cuda/9.2/prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64
Reusing existing connection to developer.nvidia.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://developer.download.nvidia.com/compute/cuda/9.2/secure/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb?VkjV1kZ6UH00CzcYRpxFwdhlEC2qqdYs5SQVb60NzKniRajGrEGiLDjug3RpLCMIs5j7T7dW-D16DQ7qHfCl16oEagCZ7xvuX0XDwc8S8TuS_

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [2]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-2_qkn1as
  Running command git clone -q https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-2_qkn1as
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4306 sha256=47d6b65b7eaf48157217a63d30e4f7a238a6714af4b0935a70788d26816dc9bd
  Stored in directory: /tmp/pip-ephem-wheel-cache-fesqivwa/wheels/ca/33/8d/3c86eb85e97d2b6169d95c6e8f2c297fdec60db6e84cb56f5e
Successfully built NVCCPlugin


In [3]:
%reload_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [10]:

%%cu
#include <cuda.h>
#include <stdio.h>
#include <math.h>
#include<sys/time.h>
#define N 6

__global__ void gaussian_reduction(float *a, int size ,int index, float * U ){
	int i;
	int tid;
  tid = threadIdx.x ;
  float b ;
  float k ;
  float f ;


  int start =  ((index+tid+1)*size+index);
  int end= ((index+tid+1)*size+size);
 

         
          f =  a [start] ;
          k = a[index*size + index];
          b = f/k;
          //*printf("valeur de b pour id : %f %d  f= %f k = %f %d \n" , b , tid , f, k , index*size);
          for(i= start;i< end;i++){
               a[i]=a[i]- b*a[(index*size)+ (index + (i-start))];
           
        } 
}


int main(){

  float result[N][N]; float *a ; float c[N*N]; float d[N*N];
  float *dev_a, *dev_b, *dev_c;
	int i; int j; int k;
	float l1; float u1; float b[N][N];  float tab[N][N];
  cudaEvent_t start, stop;
  float elapsedTime;
  cudaEventCreate(&start);
  cudaEventRecord(start,0);
  a=(float *)malloc(sizeof(float)*N*N);
  cudaMalloc ( (void**)&dev_a, N*N* sizeof (float) );
	cudaMalloc ( (void**)&dev_b, N*N* sizeof (float) );
  cudaMalloc ( (void**)&dev_c, N*N* sizeof (float) );
  srand(time(0));
  int indice = 0 ;
  printf("**************************** A ****************************** \n");	
	for ( i = 0; i < N; i++) 
 {
    for ( j=0 ; j< N ; j++) {
		tab[i][j] =rand()%1000;
    a[indice] = tab[i][j] ;
    printf("%f  " , tab[i][j]);
    indice ++ ;
	}
  printf("\n");
 }
  cudaMemcpy( dev_a, a, N*N*sizeof(float), cudaMemcpyHostToDevice);//copy array to device memory
  	for(i=0;i<N;i++){
       gaussian_reduction<<<1,N-1-i>>>(dev_a,N,i ,dev_c );
  }
  cudaDeviceSynchronize ();
  cudaEventCreate(&stop);
  cudaEventRecord(stop,0);
  cudaEventSynchronize(stop);
  cudaEventElapsedTime(&elapsedTime, start,stop);


  cudaMemcpy( &c, dev_a, N*N*sizeof(float),cudaMemcpyDeviceToHost );//copy array back to host
  
  printf("**************************** U ****************************** \n");
         for(i=0;i<N;i++){
             for(j=0;j<N;j++){
                 if (abs (c[i*N + j]) < 0.005) c[i*N + j] = 0 ;
    printf("%f " , c[i*N + j]);
		result[i][j]=c[i*N+j];
    
		}
    printf("\n");
        }
  printf("\nOrdre de la matrice : %d \n" , N); 
  printf("Le temps parallèle avec CUDA pour le calcul de la décomposition gaussiene de la matrice : %f s \n" ,elapsedTime/1000);



	//free the memory allocated on the GPU
	cudaFree( dev_a );
	cudaFree( dev_b );
	cudaFree( dev_c );
	

  
	return 0;
}

**************************** A ****************************** 
455.000000  642.000000  141.000000  60.000000  83.000000  448.000000  
685.000000  141.000000  873.000000  402.000000  329.000000  927.000000  
363.000000  206.000000  366.000000  758.000000  643.000000  837.000000  
977.000000  317.000000  811.000000  601.000000  329.000000  581.000000  
620.000000  993.000000  389.000000  902.000000  598.000000  816.000000  
825.000000  405.000000  810.000000  966.000000  817.000000  894.000000  
**************************** U ****************************** 
455.000000 642.000000 141.000000 60.000000 83.000000 448.000000 
0.000000 -825.527466 660.725281 311.670319 204.043961 252.538483 
0.000000 0.000000 8.446169 594.533020 501.102295 385.917786 
0.000000 0.000000 0.000000 24101.535156 20142.212891 14892.519531 
0.000000 0.000000 0.000000 0.000000 -354.891266 -932.884094 
0.000000 0.000000 0.000000 0.000000 0.000000 -461.681854 

Ordre de la matrice : 6 
Le temps parallèle avec CUDA pour 

In [6]:
%%writefile first.c
#include <stdio.h>
#include <stdlib.h>
#include <time.h>
#include <math.h>
#include <omp.h>
#define N 1000
#define n_threads 6


void print_matrix(float  matrix[N][N] , int size)
{

for (int i = 0; i < size; i++)
{

for (int j = 0; j < size; j++)
{
      printf("%f " , matrix[i][j]);
}

printf("\n");
}
}


void gaussian(float a[N][N], int size)
{
float l[N];
 
#pragma omp parallel for schedule (static)
for (int i = 0; i < size; i++)
{
for (int j = i ; j < size; j++)
{
		
     //* printf(" a [j][0] %f a[i][0] %f \n", a[j][0] , a[i][0]);
     if (i!=j) l[j] = a[j][i] / a[i][i];
      //*printf("%f \n %d" , l[j] , j);
      for (int k =0 ; k < size ; k++)
     {
        if (j == i) a[j][k] = a[i][k];
      else {
          if ( k < j) a[j][k] = 0 ;
          else a[j][k] = a[j][k] - l[j] * a[i][k];
           
      }
      //*printf("%f  " , a[j][k]);
         //*printf("\n");  
     }  
}
}

}





void random_fill(float matrix[N][N], int size)
{

srand(time(0));
for (int i = 0; i < size; i++)
{
for (int j = 0; j < size; j++)
{
matrix[i][j] = ((rand()%20)+1) ;
}
}

}


int main(void )
{

double runtime;
int size = N ;
srand(time(0));
// set le nombre de threads a utiliser
omp_set_num_threads(n_threads);
float a [N][N];
random_fill(a, size);
//*printf("*********************************A*************************\n");
//*print_matrix(a, size);
runtime = omp_get_wtime();
gaussian(a, size);
runtime = omp_get_wtime() - runtime;

//*printf("*********************************U*************************\n");
/*print_matrix(a, size);
 printf("\nOrdre de la matrice : %d \n" , N);
 printf("\nNombre de Threads : %d \n" , n_threads);
 printf("\nLe temps parallele avec openmp pour le calcul de la décomposition gaussiene de la matrice %f s.\n", runtime); //output time
	return 0 ;
}

Writing first.c


In [7]:
%%shell
gcc lu.c -o lu 
./lu


Ordre de la matrice = 250 
 
Le temps séquentiel pour le calcul de la décomposition gaussiene de la matrice 0.023295 s.